# Price Momentum Strategy

The crux of this strat is the hypothesis that historically well performing stocks will tend to do so in the future as well.

The criteria then falls on finding the reliable standard by which to find these stocks.

In case of dollar neutral stocks, the point is also to determine whether wealth allocation into low performing stock through shorting is favourable or not, as well.

This cell imports all the necessary libraries for the price momentum strategy, including libraries for data manipulation (pandas, numpy), plotting (matplotlib, seaborn), date calculations (dateutil.relativedelta, datetime), fetching financial data (yfinance), interacting with Google Sheets (google.colab.sheets), and portfolio optimization (scipy.cluster.hierarchy, scipy.spatial.distance, pypfopt).

In [7]:
pip install PyPortfolioOpt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 10.4 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sb
from dateutil.relativedelta import relativedelta
import yfinance as yf
pd.set_option('display.max_rows', None)
from datetime import datetime
from math import log
from google.colab import sheets
pd.set_option('future.no_silent_downcasting', True)
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import squareform
from pypfopt import HRPOpt

This cell defines the list of stock tickers that will be used in the analysis and sets the holding and skip periods for the strategy in months.

In [9]:
tickers = ["ADANIPORTS.NS", "ASIANPAINT.NS", "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJFINANCE.NS",
    "BAJAJFINSV.NS", "BPCL.NS", "BHARTIARTL.NS", "BRITANNIA.NS", "CIPLA.NS",
    "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS", "EICHERMOT.NS", "GRASIM.NS",
    "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS",
    "HINDUNILVR.NS", "ICICIBANK.NS", "ITC.NS", "INDUSINDBK.NS", "INFY.NS",
    "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS", "M&M.NS", "MARUTI.NS",
    "NTPC.NS", "NESTLEIND.NS", "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS",
    "SBILIFE.NS", "SHREECEM.NS", "SBIN.NS", "SUNPHARMA.NS", "TCS.NS",
    "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TECHM.NS", "TITAN.NS",
    "ULTRACEMCO.NS", "UPL.NS", "WIPRO.NS", "ZEEL.NS"]

holding_period = 1
skip_period = 1

This cell defines three functions to calculate different return metrics for a given ticker over a specified period:
- `cum_returns`: Calculates the cumulative returns.
- `mean_returns`: Calculates the mean monthly returns.
- `risk_adj_returns`: Calculates the risk-adjusted returns (Sharpe Ratio).
It then initializes an empty DataFrame `return_params` to store these calculated metrics for each ticker and date, and finally populates this DataFrame by iterating through the defined tickers and a range of rolling dates.

In [10]:
# Define a function to calculate cumulative returns for a given ticker over a specified period
def cum_returns(ticker, form_period, skip_period, rolling_date, holding_period):
  start = (datetime.today() - relativedelta(months= form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
  end = (datetime.today()-relativedelta(months = rolling_date-skip_period+holding_period)).strftime('%Y-%m-%d')
  price_action = yf.download(ticker, start, end, auto_adjust= True, progress=False)
  price_df = pd.DataFrame(price_action[('Close', ticker)])
  price_df.rename(columns = {ticker: 'Close'}, inplace = True)

  P_S = price_df.iat[-2, 0]
  P_ST = price_df.iat[0, 0]
  log_returns = (P_S/P_ST-1)*100

  return log_returns

def mean_returns(ticker, form_period, skip_period, rolling_date, holding_period):
  start = (datetime.today() - relativedelta(months= form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
  end = (datetime.today()-relativedelta(months = rolling_date-skip_period+holding_period)).strftime('%Y-%m-%d')
  price_action = yf.download(ticker, start, end, interval = '1mo', auto_adjust= True, progress=False)

  returns = np.log(price_action['Close'] / price_action['Close'].shift(1)).dropna()*100
  returns.rename(columns = {ticker: 'Returns'}, inplace = True)

  return returns['Returns'].mean()

def risk_adj_returns(ticker, form_period, skip_period, rolling_date, holding_period):
  start = (datetime.today() - relativedelta(months= form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
  end = (datetime.today()-relativedelta(months = rolling_date-skip_period+holding_period)).strftime('%Y-%m-%d')
  price_action = yf.download(ticker, start, end, interval = '1mo', auto_adjust= True, progress=False)

  returns = np.log(price_action['Close'] / price_action['Close'].shift(1)).dropna()
  returns.rename(columns = {ticker: 'Returns'}, inplace = True)
  mean_returns = returns['Returns'].mean()
  std_returns = returns['Returns'].std()

  return mean_returns/std_returns

def volatility_trend(ticker, form_period, skip_period, rolling_date, holding_period):
    start = (datetime.today() - relativedelta(months=form_period + rolling_date + holding_period)).strftime('%Y-%m-%d')
    mid = (datetime.today() - relativedelta(months=form_period//2 + rolling_date + holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today() - relativedelta(months=rolling_date - skip_period + holding_period)).strftime('%Y-%m-%d')

    data = yf.download(ticker, start=start, end=end, interval='1mo', auto_adjust=True, progress=False)['Close'].dropna()
    if len(data) < 4:
        return 0

    returns = np.log(data / data.shift(1)).dropna()
    half = len(returns) // 2
    first_half_vol = returns.iloc[:half].std()
    second_half_vol = returns.iloc[half:].std()
    #print(first_half_vol, second_half_vol)
    vol_val = ((second_half_vol - first_half_vol) / first_half_vol) * 100 if first_half_vol.any() != 0 else 0
    return vol_val.values


return_params = pd.DataFrame(columns = ['Date','Ticker', 'Cumulative Returns', 'Mean Returns', 'Risk Adjusted Returns', 'Volatility Trend'])

def return_params_df(tickers, form_period, skip_period, rolling_date, holding_period):
  for ticker in tickers:
    cum_returns_val = cum_returns(ticker, form_period, skip_period, rolling_date, holding_period)
    mean_returns_val = mean_returns(ticker, form_period, skip_period, rolling_date, holding_period)
    risk_adj_returns_val = risk_adj_returns(ticker, form_period, skip_period, rolling_date, holding_period)
    volatility_trend_val = volatility_trend(ticker, form_period, skip_period, rolling_date, holding_period)

    return_params.loc[len(return_params.index)] = [(datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d'),
                                                   ticker, cum_returns_val, mean_returns_val, risk_adj_returns_val, volatility_trend_val]
  return return_params

for rolling_date in range(12):
  return_params_df(tickers, 12, 1, rolling_date, 1)



This cell iterates through the rolling dates and identifies the top and bottom 5 performing stocks based on the calculated cumulative returns, mean returns, and risk-adjusted returns from the `return_params` DataFrame. It then prints the tickers of the top and bottom performing stocks based on cumulative returns for the last rolling date.

In [11]:
# Iterate through rolling dates to find top and bottom performing stocks based on different metrics
for rolling_date in range(12):
  top_by_cumret = return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Cumulative Returns', ascending = False).head()
  top_by_meanret = return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Mean Returns', ascending = False).head()
  top_by_radret = return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Risk Adjusted Returns', ascending = False).head()
  top_by_voltrend = return_params[return_params['Date'] == (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d')].sort_values(by = 'Volatility Trend', ascending = False).head()

  bottom_by_cumret = return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Cumulative Returns', ascending = True).head()
  bottom_by_meanret = return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Mean Returns', ascending = True).head()
  bottom_by_radret = return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Risk Adjusted Returns', ascending = True).head()
  bottom_by_voltrend = return_params[return_params['Date'] == (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d')].sort_values(by = 'Volatility Trend', ascending = True).head()


This cell defines two functions:
- `calculate_weights`: Calculates portfolio weights based on the inverse volatility of each stock.
- `backtest_cum_returns`: Calculates the backtested cumulative returns for a portfolio of stocks using the inverse volatility weighting scheme.
It also initializes an `initial_cap` variable, although it is not currently used in the provided code.

In [12]:
initial_cap =10000

def calculate_weights(tickers, form_period, skip_period, rolling_date, holding_period):
  weights = np.array([])
  for ticker in tickers:
    start = (datetime.today() - relativedelta(months= form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today()-relativedelta(months = rolling_date-skip_period-holding_period)).strftime('%Y-%m-%d')
    price_action = yf.download(ticker, start, end, interval = '1mo', auto_adjust= True, progress=False)
    price_df = pd.DataFrame(price_action[('Close', ticker)])

    returns = np.log(price_action['Close'] / price_action['Close'].shift(1)).dropna()
    returns.rename(columns = {ticker: 'Returns'}, inplace = True)
    std_returns = returns['Returns'].std()

    weights = np.append(weights, 1/std_returns)
  weight_n = weights/sum(weights)
  return weight_n

def backtest_cum_returns(tickers,form_period, skip_period, rolling_date, holding_period):
  returns = []
  for ticker in tickers:
    start = (datetime.today() - relativedelta(months= skip_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today() - relativedelta(months= rolling_date+skip_period)).strftime('%Y-%m-%d')
    price_action = yf.download(ticker, start, end, auto_adjust= True, progress=False)
    price_df = pd.DataFrame(price_action[('Close', ticker)])
    returns.append((price_df.iat[-1, 0]/price_df.iat[0, 0]-1)*100)
  return sum(calculate_weights(tickers, 12, 1, rolling_date, 1)*returns)

# Create trade sheet using cumulative returns

This cell creates a DataFrame `trades_sheet_df` to record the trade details for a long-only strategy based on cumulative returns. It iterates through rolling dates, selects the top performing stocks based on cumulative returns, calculates their weights using the `calculate_weights` function, and then records trade information such as buy/sell prices, returns, drawdown, upside, and weights for each selected stock.

In [27]:
#Create trade sheet using cumulative returns
trades_sheet_df = pd.DataFrame(columns = ['Year','Month','stock','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])
for rolling_date in range(12):
  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')

  stocks = return_params[return_params['Date'] == (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d')].sort_values(by = 'Cumulative Returns', ascending = False).head()['Ticker'].values
  weight = calculate_weights(stocks, 12, 1, rolling_date, 1)
  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]

  for stock in stocks:
    if weight.all() > 0:
      position = 'Buy'
    else:
      position = 'Sell' #Safeguard, doesn't function in long only strat

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      auto_adjust= True, progress=False)

    weight_norm = weights_df.loc[weights_df['Ticker'] == stock, 'Weight']
    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return # This is not monthly volatility
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_sheet_df.loc[len(trades_sheet_df.index)] = [year, month, stock, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm.values[0]]

In [ ]:
# Commented out code to create an interactive sheet for cumulative returns trades
# This is likely commented out because the sheet has already been generated
#sheet = sheets.InteractiveSheet(title = 'Trade Sheet Based on Cumulative Returns',df=trades_sheet_df)

This cell creates a DataFrame `trades_sheet_df_m` to record the trade details for a long-only strategy based on mean returns. Similar to the cumulative returns tradesheet, it iterates through rolling dates, selects the top performing stocks based on mean returns, calculates their weights, and records trade information.

# Create trade sheet using mean returns

In [28]:
#Create trade sheet using mean returns
trades_sheet_df_m = pd.DataFrame(columns = ['Year','Month','Position','stock','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])
for rolling_date in range(12):

  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks = return_params[return_params['Date'] == (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d')].sort_values(by = 'Mean Returns', ascending = False).head()['Ticker'].values

  weight = calculate_weights(stocks, 12, 1, rolling_date, 1)

  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]

  for stock in stocks:

    if weight.all() > 0:
      position = 'Buy'
    else:
      position = 'Sell' #Safeguard, doesn't function in long only strat

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      auto_adjust= True, progress=False)
    weight_norm = weights_df.loc[weights_df['Ticker'] == stock, 'Weight']

    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return # This is not monthly volatility
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_sheet_df_m.loc[len(trades_sheet_df_m.index)] = [year, month,position, stock, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm.values[0]]

In [ ]:
# Commented out code to create an interactive sheet for mean returns trades
# This is likely commented out because the sheet has already been generated
#sheet = sheets.InteractiveSheet(title = 'Trade Sheet Based on Mean Returns',df=trades_sheet_df_m)

# Create trade sheet using risk adjusted returns

This cell creates a DataFrame `trades_sheet_df_r` to record the trade details for a long-only strategy based on risk-adjusted returns. It iterates through rolling dates, selects the top performing stocks based on risk-adjusted returns, calculates their weights, and records trade information.

In [29]:
trades_sheet_df_r = pd.DataFrame(columns = ['Year','Month','Position','stock','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])
for rolling_date in range(12):

  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks = return_params[return_params['Date'] == (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d')].sort_values(by = 'Risk Adjusted Returns', ascending = False).head()['Ticker'].values
  weight = calculate_weights(stocks, 12, 1, rolling_date, 1)
  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]

  for stock in stocks:
    if weight.all() > 0:
      position = 'Buy'
    else:
      position = 'Sell' #Safeguard, doesn't function in long only strat

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      auto_adjust= True, progress=False)
    weight_norm = weights_df.loc[weights_df['Ticker'] == stock, 'Weight']
    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return # This is not monthly volatility
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_sheet_df_r.loc[len(trades_sheet_df_r.index)] = [year, month,position, stock, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm.values[0]]

In [ ]:
# Commented out code to create an interactive sheet for risk adjusted returns trades
# This is likely commented out because the sheet has already been generated
#sheet = sheets.InteractiveSheet(title = 'Trade Sheet Based on Risk Adjusted Returns', df=trades_sheet_df_r)

# Create trade sheet using Volatility Trend


This cell creates a DataFrame trades_sheet_df_v to record the trade details for a long-only strategy based on volatility trend. It iterates through rolling dates, selects the top performing stocks based on volatility trend, calculates their weights, and records trade information.

In [13]:
#Create trade sheet using volatility returns
trades_sheet_df_v = pd.DataFrame(columns = ['Year','Month','Position','stock','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])
for rolling_date in range(12):

  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks = return_params[return_params['Date'] == (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d')].sort_values(by = 'Volatility Trend', ascending = False).head()['Ticker'].values

  weight = calculate_weights(stocks, 12, 1, rolling_date, 1)

  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]

  for stock in stocks:

    if weight.all() > 0:
      position = 'Buy'
    else:
      position = 'Sell' #Safeguard, doesn't function in long only strat

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                      auto_adjust= True, progress=False)
    weight_norm = weights_df.loc[weights_df['Ticker'] == stock, 'Weight']

    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return # This is not monthly volatility
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_sheet_df_v.loc[len(trades_sheet_df_v.index)] = [year, month,position, stock, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm.values[0]]

In [52]:
sheet = sheets.InteractiveSheet(title = 'Trade Sheet Based on Volatility Trend', df=trades_sheet_df_v)

https://docs.google.com/spreadsheets/d/1WlMJhccjwjJS2sYgkKk6X6078iezB02hj7TgmMIQslE/edit#gid=0


## Calculates returns over all instances of holding periods, and returns max returns and the factor used to gauge that momentum

In [30]:
def returns_performace(trades_sheet_df):
  returns = np.array(trades_sheet_df['Monthly_Return'].dropna())
  weights = np.array(trades_sheet_df['Weight_norm'].dropna())
  weighted_ret = returns@weights
  return weighted_ret

performance_df = pd.DataFrame(columns = ['Cumulative Returns', 'Mean Returns', 'Risk Adjusted Returns', 'Volatility Trend'])
performance_df.loc[len(performance_df.index)] = [returns_performace(trades_sheet_df), returns_performace(trades_sheet_df_m), returns_performace(trades_sheet_df_r), returns_performace(trades_sheet_df_v)]

print("Max Returns Using: ", performance_df.iloc[0].idxmax())
print("Max Returns: ", performance_df.iloc[0].max())

Max Returns Using:  Mean Returns
Max Returns:  36.52329284926282


# Generates and Updates Individual Stock's Sheet

In [ ]:
# Commented out code to generate and update individual stock sheets based on the highest performing metric
'''if performance_df.iloc[0].idxmax() == 'Cumulative Returns':
  for stock in trades_sheet_df['stock']:
    individual_sheet = trades_sheet_df[trades_sheet_df['stock'] == stock]
    sheet = sheets.InteractiveSheet(title=stock+' LONG ONLY',df=individual_sheet)
elif performance_df.iloc[0].idxmax() == 'Mean Returns':
  for stock in trades_sheet_df_m['stock']:
    individual_sheet = trades_sheet_df_m[trades_sheet_df_m['stock'] == stock]
    sheet = sheets.InteractiveSheet(title=stock+' LONG ONLY',df=individual_sheet)
elif performance_df.iloc[0].idxmax() == 'Risk Adjusted Returns':
  for stock in trades_sheet_df_r['stock']:
    individual_sheet = trades_sheet_df_r[trades_sheet_df_r['stock'] == stock]
    sheet = sheets.InteractiveSheet(title=stock+' LONG ONLY',df=individual_sheet)
elif performance_df.iloc[0].idxmax() == 'Volatility Trend':
  for stock in trades_sheet_df_v['stock']:
    individual_sheet = trades_sheet_df_v[trades_sheet_df_v['stock'] == stock]'''

"if performance_df.iloc[0].idxmax() == 'Cumulative Returns':\n  for stock in trades_sheet_df['stock']:\n    individual_sheet = trades_sheet_df[trades_sheet_df['stock'] == stock]\n    sheet = sheets.InteractiveSheet(title=stock+' LONG ONLY',df=individual_sheet)\nelif performance_df.iloc[0].idxmax() == 'Mean Returns':\n  for stock in trades_sheet_df_m['stock']:\n    individual_sheet = trades_sheet_df_m[trades_sheet_df_m['stock'] == stock]\n    sheet = sheets.InteractiveSheet(title=stock+' LONG ONLY',df=individual_sheet)\nelif performance_df.iloc[0].idxmax() == 'Risk Adjusted Returns':\n  for stock in trades_sheet_df_r['stock']:\n    individual_sheet = trades_sheet_df_r[trades_sheet_df_r['stock'] == stock]\n    sheet = sheets.InteractiveSheet(title=stock+' LONG ONLY',df=individual_sheet)"

# LONG SHORT STRAT

Calculates weights for long and short stocks through Hierarchial Risk Parity. The original version uses inverse variance.

In [14]:
def calculate_weights_LongShort(long_tickers, short_tickers, form_period, skip_period, rolling_date, holding_period=1):
  weights_l = np.array([])
  weights_s = np.array([])
  weights = np.array([])
  for ticker in long_tickers:
    start = (datetime.today() - relativedelta(months= form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today()-relativedelta(months = rolling_date-skip_period-holding_period)).strftime('%Y-%m-%d')
    price_action_l = yf.download(ticker, start, end, interval = '1mo', auto_adjust= True, progress=False)


    price_df_l = pd.DataFrame(price_action_l[('Close', ticker)])

    returns_l = np.log(price_action_l['Close'] / price_action_l['Close'].shift(1)).dropna()
    returns_l.rename(columns = {ticker: 'Returns_l'}, inplace = True)
    std_returns_l = returns_l['Returns_l'].std()

    weights_l = np.append(weights_l, 1/std_returns_l)

  for ticker in short_tickers:
    start = (datetime.today() - relativedelta(months= form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today()-relativedelta(months = rolling_date-skip_period-holding_period)).strftime('%Y-%m-%d')
    price_action_s = yf.download(ticker, start, end, interval = '1mo', auto_adjust= True, progress=False)
    price_df_s = pd.DataFrame(price_action_s[('Close', ticker)])

    returns_s = np.log(price_action_s['Close'] / price_action_s['Close'].shift(1)).dropna()
    returns_s.rename(columns = {ticker: 'Returns_s'}, inplace = True)
    std_returns_s = -returns_s['Returns_s'].std()

    weights_s = np.append(weights_s, 1/std_returns_s)

  weight_n_l = weights_l/sum(weights_l)
  weight_n_s = -weights_s/sum(weights_s)
  weight_n = np.append(weight_n_l, weight_n_s)
  return weight_n

# Define a function to calculate weights for long and short positions using Hierarchical Risk Parity (HRP)
def calculate_weights_LongShort_HRP(long_tickers, short_tickers, form_period, skip_period, rolling_date, holding_period=1):
    base_date = datetime.today()

    all_tickers = list(long_tickers) + list(short_tickers)
    price_data = pd.DataFrame()

    for ticker in all_tickers:
        start = (base_date - relativedelta(months=form_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
        end = (base_date - relativedelta(months=rolling_date-skip_period-holding_period)).strftime('%Y-%m-%d')

        df = yf.download(ticker, start, end, interval='1mo', auto_adjust=True, progress=False)
        if not df.empty:
            price_data[ticker] = df['Close']

    price_data = price_data.dropna(axis=0, how='any')

    if price_data.shape[0] < 2:
        return np.zeros(len(all_tickers))

    returns = price_data.pct_change().dropna()

    cov = returns.cov()

    corr = returns.corr()

    dist = np.sqrt(0.5 * (1 - corr))

    link = linkage(squareform(dist), method='single')

    def get_quasi_diag(link):
        link = link.astype(int)
        sort_ix = pd.Series([link[-1, 0], link[-1, 1]])
        num_items = link[-1, 3]
        while sort_ix.max() >= num_items:
            sort_ix.index = range(0, sort_ix.shape[0]*2, 2)
            df0 = sort_ix[sort_ix >= num_items]
            i = df0.index
            j = df0.values - num_items
            sort_ix[i] = link[j, 0]
            df1 = pd.Series(link[j, 1], index=i+1)
            sort_ix = pd.concat([sort_ix, df1])
            sort_ix = sort_ix.sort_index()
        return sort_ix.tolist()

    sort_ix = get_quasi_diag(link)
    sorted_tickers = corr.columns[sort_ix]

    def get_rec_bipart(cov, sort_ix):
        weights = pd.Series(1.0, index=sort_ix)
        cluster_items = [sort_ix]

        while len(cluster_items) > 0:
            cluster_items = [i[j:k] for i in cluster_items for j,k in ((0, len(i)//2), (len(i)//2, len(i))) if len(i) > 1]
            for cluster in cluster_items:
                if len(cluster) <= 1:
                    continue
                cov_slice = cov.loc[cluster, cluster]
                inv_diag = 1/np.diag(cov_slice)
                alloc = pd.Series(inv_diag / inv_diag.sum(), index=cov_slice.index) # Ensure index is preserved
                cluster_var = (alloc * cov_slice @ alloc).sum()
                first_cluster = cluster[:len(cluster)//2]
                second_cluster = cluster[len(cluster)//2:]
                var_first = (alloc[first_cluster] * cov_slice.loc[first_cluster, first_cluster] @ alloc[first_cluster]).sum()
                var_second = (alloc[second_cluster] * cov_slice.loc[second_cluster, second_cluster] @ alloc[second_cluster]).sum()
                alpha = 1 - var_first / (var_first + var_second)
                weights[first_cluster] *= alpha
                weights[second_cluster] *= (1 - alpha)

        return weights

    hrp_weights = get_rec_bipart(cov, sorted_tickers)

    long_weights = hrp_weights[long_tickers]
    short_weights = -hrp_weights[short_tickers]

    long_weights /= long_weights.abs().sum()
    short_weights /= short_weights.abs().sum()

    final_weights = pd.concat([long_weights, short_weights])

    return final_weights.values


def backtest_cum_returns(long_tickers, short_tickers, form_period, skip_period, rolling_date, holding_period = 1):
  returns = []

  for ticker in long_tickers:
    start = (datetime.today() - relativedelta(months= skip_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today() - relativedelta(months= rolling_date+skip_period)).strftime('%Y-%m-%d')
    price_action = yf.download(ticker, start, end, auto_adjust= True, progress=False)
    price_df = pd.DataFrame(price_action[('Close', ticker)])
    returns.append((price_df.iat[-1, 0]/price_df.iat[0, 0]-1)*100)

  for ticker in short_tickers:
    start = (datetime.today() - relativedelta(months= skip_period+rolling_date+holding_period)).strftime('%Y-%m-%d')
    end = (datetime.today() - relativedelta(months= rolling_date+skip_period)).strftime('%Y-%m-%d')
    price_action = yf.download(ticker, start, end, auto_adjust= True, progress=False)
    price_df = pd.DataFrame(price_action[('Close', ticker)])
    returns.append(-(price_df.iat[-1, 0]/price_df.iat[0, 0]-1)*100)

  return sum(calculate_weights_LongShort_HRP(long_tickers, short_tickers, form_period, skip_period, rolling_date, holding_period)*returns)

In [15]:
# Calculate and print the backtested returns for the long-short strategy using different ranking metrics
print("Returns by Cumulative Stocks: ", backtest_cum_returns(top_by_cumret['Ticker'].values, bottom_by_cumret['Ticker'].values, 12, 1, 0, 1))
print("Returns by Mean Stocks: ", backtest_cum_returns(top_by_meanret['Ticker'].values, bottom_by_meanret['Ticker'].values, 12, 1, 0, 1))
print("Returns by Risk Adjusted Stocks: ", backtest_cum_returns(top_by_radret['Ticker'].values, bottom_by_radret['Ticker'].values, 12, 1, 0, 1))

Returns by Cumulative Stocks:  0.0826180792397605
Returns by Mean Stocks:  9.438206667514194
Returns by Risk Adjusted Stocks:  2.873169283462919


In [16]:
#Create trade sheet using Cumulative Returns
trades = []
for rolling_date in range(0,12):
  ticker_list = pd.concat([return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Cumulative Returns', ascending = False).head(),
                                  return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Cumulative Returns', ascending = True).head()])

  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks = ticker_list['Ticker'].values
  weight = calculate_weights_LongShort_HRP(top_by_cumret['Ticker'].values, bottom_by_cumret['Ticker'].values, 12, 1, rolling_date, 1)/2
  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]
  for stock in stocks:
    for index, row in weights_df[weights_df['Ticker']==stock].iterrows():
      weight_norm = row['Weight']
      if weight_norm > 0:
        position = 'Buy'
      else:
        position = 'Sell'

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        auto_adjust= True, progress=False)

    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades.append([year, month, stock, position, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm])

trades_sheet_df_ls = pd.DataFrame(trades, columns = ['Year','Month','stock','position','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])

In [ ]:
# Create an interactive sheet for the long-short cumulative returns trades
sheet = sheets.InteractiveSheet(title = 'LONG SHORT: Trade Sheet Based on Cumulative Returns', df=trades_sheet_df_ls)

https://docs.google.com/spreadsheets/d/1Q4iUkvdu4GyD7S2egU7iq4v82BNf_CsRQObm_ywyhUc/edit#gid=0


In [17]:
#Create trade sheet using mean returns
trades_m = []
for rolling_date in range(12):
  ticker_list_m = pd.concat([return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Mean Returns', ascending = False).head(),
                                  return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Mean Returns', ascending = True).head()])
  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks_m = ticker_list_m['Ticker'].values
  weight = calculate_weights_LongShort_HRP(top_by_meanret['Ticker'].values, bottom_by_meanret['Ticker'].values, 12, 1, rolling_date, 1)/2
  weights_df_m = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks_m)):
    weights_df_m.loc[len(weights_df_m.index)] = [stocks_m[i], weight[i]]
  for stock in stocks_m:
    for index, row in weights_df_m[weights_df_m['Ticker']==stock].iterrows():
      weight_norm = row['Weight']
      if weight_norm > 0:
        position = 'Buy'
      else:
        position = 'Sell'

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        auto_adjust= True, progress=False)

    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return # This is not monthly volatility
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_m.append([year, month, stock, position, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm])

trades_sheet_df_ls_m = pd.DataFrame(trades_m, columns = ['Year','Month','stock','position','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])

In [ ]:
# Create an interactive sheet for the long-short mean returns trades
sheet = sheets.InteractiveSheet(title = 'LONG SHORT: Trade Sheet Based on Mean Returns', df=trades_sheet_df_ls_m)

https://docs.google.com/spreadsheets/d/1piCWQqquSbyBVVfA-_LTL_xMbASoQcUSs8hgrba8Lmo/edit#gid=0


In [18]:
#Create trade sheet using Risk Adjusted Returns
trades_rad = []
for rolling_date in range(12):
  ticker_list = pd.concat([return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Risk Adjusted Returns', ascending = False).head(),
                                  return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Risk Adjusted Returns', ascending = True).head()])
  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks = ticker_list['Ticker'].values
  weight = calculate_weights_LongShort_HRP(top_by_radret['Ticker'].values, bottom_by_radret['Ticker'].values, 12, 1, rolling_date, 1)/2
  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]
  for stock in stocks:
    for index, row in weights_df[weights_df['Ticker']==stock].iterrows():
      weight_norm = row['Weight']
      if weight_norm > 0:
        position = 'Buy'
      else:
        position = 'Sell'

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        auto_adjust= True, progress=False)

    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_rad.append([year, month, stock, position, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm])

trades_sheet_df_ls_rad = pd.DataFrame(trades, columns = ['Year','Month','stock','position','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])

In [ ]:
# Create an interactive sheet for the long-short risk adjusted returns trades
sheet = sheets.InteractiveSheet(title = 'LONG SHORT: Trade Sheet Based on Risk Adjusted Returns', df=trades_sheet_df_ls_rad)

https://docs.google.com/spreadsheets/d/1FiuhzxOfK56GDpv0ZbcO_flGFFLpHNoa7x-uV51SxCo/edit#gid=0


In [21]:
#Create trade sheet using Volatility Trends
trades_v = []
for rolling_date in range(0,12):
  ticker_list = pd.concat([return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Volatility Trend', ascending = False).head(),
                                  return_params[return_params['Date'] == (datetime.today()-relativedelta(months = rolling_date+skip_period+holding_period)).strftime('%Y-%m-%d')].sort_values(by = 'Volatility Trend', ascending = True).head()])

  year = (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y')
  month = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%m')
  stocks = ticker_list['Ticker'].values
  weight = calculate_weights_LongShort_HRP(top_by_voltrend['Ticker'].values, bottom_by_voltrend['Ticker'].values, 12, 1, rolling_date, 1)/2
  weights_df = pd.DataFrame(columns = ['Ticker','Weight'])

  for i in range(len(stocks)):
    weights_df.loc[len(weights_df.index)] = [stocks[i], weight[i]]
  for stock in stocks:
    for index, row in weights_df[weights_df['Ticker']==stock].iterrows():
      weight_norm = row['Weight']
      if weight_norm > 0:
        position = 'Buy'
      else:
        position = 'Sell'

    price_data = yf.download(stock, (datetime.today() - relativedelta(months= holding_period+skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        end = (datetime.today() - relativedelta(months= skip_period+rolling_date)).strftime('%Y-%m-%d'),
                        auto_adjust= True, progress=False)

    drawdown = (1+price_data['Close'].pct_change()).cumprod()/((1+price_data['Close'].pct_change()).cumprod()).cummax()-1
    upside = (1+price_data['Close'].pct_change()).max()
    buy = price_data.iloc[0,0]
    sell = price_data.iloc[-1,0]
    monthly_return = log(price_data.iat[-1, 0]/price_data.iat[0, 0])*100
    cumulative_return = cum_returns(stock, 12, 1, rolling_date, 1)
    mean_monthly_return = mean_returns(stock, 12, 1, rolling_date, 1)
    monthly_volatility = 1/risk_adj_returns(stock, 12, 1, rolling_date, 1)*mean_monthly_return
    risk_adj_mean_return = risk_adj_returns(stock, 12, 1, rolling_date,1)

    trades_v.append([year, month, stock, position, buy, sell, drawdown[stock].min(), upside[stock].max(), monthly_return, cumulative_return, mean_monthly_return, monthly_volatility, risk_adj_mean_return, weight_norm])

trades_sheet_df_ls_v = pd.DataFrame(trades_v, columns = ['Year','Month','stock','position','buy','sell', 'drawdown', 'upside', 'Monthly_Return', 'Cumulative_return', 'Mean_Monthly_Return', 'Monthly_Volatility', 'Risk_Adjusted_Mean_Return', 'Weight_norm'])

In [22]:
# Create an interactive sheet for the long-short risk adjusted returns trades
sheet = sheets.InteractiveSheet(title = 'LONG SHORT: Trade Sheet Based on Volatility Trend', df=trades_sheet_df_ls_v)

https://docs.google.com/spreadsheets/d/1MfkrzlzjJGw1wcVbcweoH4oNVuMSTMwvDKJZQWeoM3U/edit#gid=0


In [25]:
def returns_performance(trades_sheet_df):
  returns = np.array(trades_sheet_df['Monthly_Return'].dropna())
  weights = np.array(trades_sheet_df['Weight_norm'].dropna())

  weighted_ret = returns@weights
  return weighted_ret

performance_df_longshort = pd.DataFrame(columns = ['Cumulative Returns', 'Mean Returns', 'Risk Adjusted Returns', 'Volatility Trend'])

performance_df_longshort.loc[len(performance_df_longshort.index)] = [returns_performance(trades_sheet_df_ls), returns_performance(trades_sheet_df_ls_m), returns_performance(trades_sheet_df_ls_rad), returns_performance(trades_sheet_df_ls_v)]

print("Max Returns Using: ", performance_df_longshort.iloc[0].idxmax())
print("Max Returns: ", performance_df_longshort.iloc[0].max())

Max Returns Using:  Mean Returns
Max Returns:  23.420302323085053


In [ ]:
# Checks for the highest performing long-short metric and produces individual sheets based on that. Commented now because sheets already generated.
'''if performance_df_longshort.iloc[0].idxmax() == 'Cumulative Returns':
  for stock in trades_sheet_df_ls['stock']:
    individual_sheet = trades_sheet_df_ls[trades_sheet_df_ls['stock'] == stock]
    sheet = sheets.InteractiveSheet(title=stock+' LONG SHORT',df=individual_sheet)
elif performance_df_longshort.iloc[0].idxmax() == 'Mean Returns':
  for stock in trades_sheet_df_ls_m['stock']:
    individual_sheet = trades_sheet_df_ls_m[trades_sheet_df_ls_m['stock'] == stock]
    sheet = sheets.InteractiveSheet(title=stock+' LONG SHORT',df=individual_sheet)
elif performance_df_longshort.iloc[0].idxmax() == 'Risk Adjusted Returns':
  for stock in trades_sheet_df_ls_rad['stock']:
    individual_sheet = trades_sheet_df_ls_rad[trades_sheet_df_ls_rad['stock'] == stock]
    sheet = sheets.InteractiveSheet(title=stock+' LONG SHORT',df=individual_sheet)
elif performance_df_longshort.iloc[0].idxmax() == 'Volatility Trend':
  for stock in trades_sheet_df_ls_v['stock']:
    individual_sheet = trades_sheet'''

# Portfolio Sheet Long Only

In [31]:
portfolio_rows = []

if performance_df.iloc[0].idxmax() == 'Cumulative Returns':
  trades_sheet_df_final = trades_sheet_df
  text = ' Cumulative Returns'
elif performance_df.iloc[0].idxmax() == 'Mean Returns':
  trades_sheet_df_final = trades_sheet_df_m
  text = ' Mean Returns'
elif performance_df.iloc[0].idxmax() == 'Risk Adjusted Returns':
  trades_sheet_df_final = trades_sheet_df_r
  text = ' Risk Adjusted Returns'
elif performance_df.iloc[0].idxmax() == 'Volatility Trend':
  trades_sheet_df_final = trades_sheet_df_v
  text = ' Volatility Trend'

tickers = ["ADANIPORTS.NS", "ASIANPAINT.NS", "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJFINANCE.NS",
    "BAJAJFINSV.NS", "BPCL.NS", "BHARTIARTL.NS", "BRITANNIA.NS", "CIPLA.NS",
    "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS", "EICHERMOT.NS", "GRASIM.NS",
    "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS",
    "HINDUNILVR.NS", "ICICIBANK.NS", "ITC.NS", "INDUSINDBK.NS", "INFY.NS",
    "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS", "M&M.NS", "MARUTI.NS",
    "NTPC.NS", "NESTLEIND.NS", "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS",
    "SBILIFE.NS", "SHREECEM.NS", "SBIN.NS", "SUNPHARMA.NS", "TCS.NS",
    "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TECHM.NS", "TITAN.NS",
    "ULTRACEMCO.NS", "UPL.NS", "WIPRO.NS", "ZEEL.NS"]
for rolling_date in range(12):

    month = (datetime.today() - relativedelta(months=skip_period+rolling_date)).strftime('%m')
    year = (datetime.today() - relativedelta(months=skip_period+rolling_date)).strftime('%Y')

    monthly_trades = trades_sheet_df_final[(trades_sheet_df_final['Month'] == month) &(trades_sheet_df_final['Year'] == year)]
    portfolio_returns = (monthly_trades['Monthly_Return'] * monthly_trades['Weight_norm']).sum()

    row = {
        'Month': month,
        'Year': year,
        'Portfolio_Returns': portfolio_returns
    }

    for stock in tickers:
        if stock in monthly_trades['stock'].values:
            weight = monthly_trades.loc[monthly_trades['stock'] == stock, 'Weight_norm'].values[0]
        else:
            weight = 0
        row[stock] = weight
    portfolio_rows.append(row)

portfolio_returns_df = pd.DataFrame(portfolio_rows)

In [ ]:
# Commented out code to create an interactive sheet for the long-only portfolio
#sheet = sheets.InteractiveSheet(title = 'Portfolio Sheet' + ' Long Only', df=portfolio_returns_df)

# Portfolio Sheet Long Short

In [32]:
# Initialize an empty list to store portfolio performance data for the long-short strategy
portfolio_rows_ls = []
# Determine which tradesheet DataFrame to use based on the highest performing long-short metric
if performance_df_longshort.iloc[0].idxmax() == 'Cumulative Returns':
  trades_sheet_df_final_ls = trades_sheet_df_ls
  text = ' Cumulative Returns'
elif performance_df_longshort.iloc[0].idxmax() == 'Mean Returns':
  trades_sheet_df_final_ls = trades_sheet_df_ls_m
  text = ' Mean Returns'
elif performance_df_longshort.iloc[0].idxmax() == 'Risk Adjusted Returns':
  trades_sheet_df_final_ls = trades_sheet_df_ls_rad
  text = ' Risk Adjusted Returns'

# Define the list of all tickers
tickers = ["ADANIPORTS.NS", "ASIANPAINT.NS", "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJFINANCE.NS",
    "BAJAJFINSV.NS", "BPCL.NS", "BHARTIARTL.NS", "BRITANNIA.NS", "CIPLA.NS",
    "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS", "EICHERMOT.NS", "GRASIM.NS",
    "HCLTECH.NS", "HDFCBANK.NS", "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS",
    "HINDUNILVR.NS", "ICICIBANK.NS", "ITC.NS", "INDUSINDBK.NS", "INFY.NS",
    "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS", "M&M.NS", "MARUTI.NS",
    "NTPC.NS", "NESTLEIND.NS", "ONGC.NS", "POWERGRID.NS", "RELIANCE.NS",
    "SBILIFE.NS", "SHREECEM.NS", "SBIN.NS", "SUNPHARMA.NS", "TCS.NS",
    "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS", "TECHM.NS", "TITAN.NS",
    "ULTRACEMCO.NS", "UPL.NS", "WIPRO.NS", "ZEEL.NS"]
for rolling_date in range(12):
    month = (datetime.today() - relativedelta(months=skip_period+rolling_date)).strftime('%m')
    year = (datetime.today() - relativedelta(months=skip_period+rolling_date)).strftime('%Y')

    monthly_trades_ls = trades_sheet_df_final_ls[(trades_sheet_df_final_ls['Month'] == month) &(trades_sheet_df_final_ls['Year'] == year)]
    portfolio_returns = (monthly_trades_ls['Monthly_Return'] * monthly_trades_ls['Weight_norm']).sum()

    row = {
        'Month': month,
        'Year': year,
        'Portfolio_Returns': portfolio_returns
    }

    for stock in tickers:
        if stock in monthly_trades_ls['stock'].values:
            weight = monthly_trades_ls.loc[monthly_trades_ls['stock'] == stock, 'Weight_norm'].values[0]
        else:
            weight = 0
        row[stock] = weight
    portfolio_rows_ls.append(row)

portfolio_returns_df_ls = pd.DataFrame(portfolio_rows_ls)

In [ ]:
# Create an interactive sheet for the long-short portfolio
sheet = sheets.InteractiveSheet(title = 'Portfolio Sheet' + ' Long Short', df=portfolio_returns_df_ls)

https://docs.google.com/spreadsheets/d/1VpHyKOwYeWPHViu7OeQJzXHhQXMeHwVexrNfH0vdDTM/edit#gid=0
